<a href="https://colab.research.google.com/github/youssefawwad88/Tradingstation/blob/main/Trading_Station.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================================
#  MASTER SETUP CELL - Run this ONCE every time you start a new session
# ===============================================================

# 1. Connect to your permanent storage in Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Load your secret API key into the environment for our scripts to use
from google.colab import userdata
import os
os.environ['ALPHA_VANTAGE_API_KEY'] = userdata.get('ALPHA_VANTAGE_API_KEY')

# 3. Install any Python libraries that Colab doesn't have by default
!pip install schedule pytz pandas requests -q

print("\n✅ Environment setup complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Environment setup complete!


# Command: Execute the Data Fetcher
This command runs the update_all_data.py script. The script's job is to read your tickerlist.txt file, then loop through each ticker and fetch the complete daily, 30-minute, and 1-minute data from the Alpha Vantage API. It then saves this data into organized CSV files within your Google Drive.

What to Expect: You will see a series of print statements in the output as it processes each ticker and data type. This is the initial "bulk download" of your data.

In [ ]:
!python "/content/drive/MyDrive/trading-system/jobs/update_all_data.py"

✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.
--- Starting Full Market Data Update ---
Processing 8 tickers: ['NVDA', 'AAPL', 'TSLA', 'AMD', 'GOOGL', 'MSFT', 'AMZN', 'NFLX']

--- Updating data for NVDA ---
--> Fetching DAILY data for NVDA...
    [✔] Successfully fetched 6472 rows of daily data for NVDA.
    Merging data: Found 6471 existing rows in NVDA_daily.csv.
    Merge complete: Added 1 new unique row(s). Total rows: 6472.
    [✔] Successfully saved data for NVDA to NVDA_daily.csv
--> Fetching REAL-TIME 30min data for NVDA...
    [✔] Successfully fetched 666 rows for NVDA.
    Merging data: Found 672 existing rows in NVDA_30min.csv.
    Merge complete: Added 26 new unique row(s). Total rows: 698.
    [✔] Successfully saved data for NVDA to NVDA_30min.csv
--> Fetching REAL-TIME 1min data for NVDA...
    [✔] Successfully fetched 19980 rows for NVDA.
    Merging data: Found 20134 existing rows in NVDA_1min.csv.
  

## **Command: Execute the Compact Intraday Updater**
This command runs the update_intraday_compact.py script. This script is optimized for frequent, fast updates. It only downloads the last 100 data points from the API and intelligently appends the newest 1-minute and 30-minute candles to your existing CSV files.

When to Use: This is the command you will use repeatedly throughout the trading day to keep your intraday data live.

In [ ]:
!python "/content/drive/MyDrive/trading-system/jobs/update_intraday_compact.py"

✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.
--- Starting Compact Intraday Data Update ---
Processing 8 tickers for compact update...

--- Updating intraday data for NVDA ---
--> Fetching REAL-TIME 30min data for NVDA...
    [✔] Successfully fetched 100 rows for NVDA.
    Merging data: Found 698 existing rows in NVDA_30min.csv.
    Merge complete: Added 0 new unique row(s). Total rows: 698.
    [✔] Successfully saved data for NVDA to NVDA_30min.csv
--> Fetching REAL-TIME 1min data for NVDA...
    [✔] Successfully fetched 100 rows for NVDA.
    Merging data: Found 20940 existing rows in NVDA_1min.csv.
    Merge complete: Added 0 new unique row(s). Total rows: 20940.
    [✔] Successfully saved data for NVDA to NVDA_1min.csv

--- Updating intraday data for AAPL ---
--> Fetching REAL-TIME 30min data for AAPL...
    [✔] Successfully fetched 100 rows for AAPL.
    Merging data: Found 698 existing rows in AAPL_30min.csv.


#Gap & Go Screener
This command will run the gapgo.py script we just created. The script will load the data for each of your tickers, apply the Gap & Go trading logic, and print a message if it finds any valid setups. If any signals are found, it will save them to a new file: /data/signals/gapgo_signals.csv.

What to Expect: The script will print its progress for each ticker. If it's outside of market hours, it will likely skip most tickers because there is no fresh pre-market or opening data.

In [ ]:
!python "/content/drive/MyDrive/trading-system/screeners/gapgo.py"

✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.

--- Running Gap & Go Screener (Execution-Grade) ---
Current Market Session Detected: PREMARKET
Saving 8 signal(s) from gapgo to /content/drive/MyDrive/trading-system/data/signals/gapgo_signals.csv...


# 10-Minute ORB Screener
Description: This command runs the orb.py script. The script identifies stocks that break out of their first 10-minute opening range (9:30 - 9:40 AM EST). For best results, run this command at or after 9:41 AM EST. It will save any identified breakouts or breakdowns to /data/signals/orb_signals.csv.

In [ ]:
!python "/content/drive/MyDrive/trading-system/screeners/orb.py"

✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.

--- Running Opening Range Breakout (ORB) Screener ---
Market session is PREMARKET. ORB screener only runs during REGULAR session.


# Find AVWAP Anchor Points

This command runs the find_avwap_anchors.py job. It analyzes daily data to find significant "power candles" and saves them as anchor dates to /data/avwap_anchors.csv. Run this once before starting the main screener.

In [ ]:
!python "/content/drive/MyDrive/trading-system/jobs/find_avwap_anchors.py"

✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.

--- Running AVWAP Anchor Finder Job ---
Successfully saved 8 detailed anchor sets to /content/drive/MyDrive/trading-system/data/avwap_anchors.csv


# Anchored VWAP Screener

This command runs the avwap.py screener. It reads the pre-calculated anchor dates and checks for real-time reclaims or rejections of the AVWAP levels on high volume. It saves the results to /data/signals/avwap_signals.csv.

In [ ]:
!python "/content/drive/MyDrive/trading-system/screeners/avwap.py"

✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.

--- Running Anchored VWAP (AVWAP) Screener ---
Saving 8 signal(s) from avwap to /content/drive/MyDrive/trading-system/data/signals/avwap_signals.csv...


# Breakout Screener
Description: This command runs the breakout.py script. It analyzes daily data to find stocks that have closed above their upper Bollinger Band on high volume (at least 150% of the 20-day average). It saves any valid signals to /data/signals/breakout_signals.csv.

In [21]:
!python "/content/drive/MyDrive/trading-system/screeners/breakout.py"

✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.

--- Running Daily Breakout & Breakdown Screener (Advanced) ---
   - INFO: No confirmed anchor date found for AAPL. AVWAP check will default to 'No'.
   - INFO: No confirmed anchor date found for MSFT. AVWAP check will default to 'No'.
   - INFO: No confirmed anchor date found for AMZN. AVWAP check will default to 'No'.
Saving 8 signal(s) from breakout to /content/drive/MyDrive/trading-system/data/signals/breakout_signals.csv...


# EMA Trend Pullback Screener

Description: This command runs the ema_pullback.py script. It analyzes daily data to find stocks in a clear trend (above/below the 50 EMA) that have pulled back to the 21 EMA and then printed a strong confirmation candle on high volume. It saves any valid signals to /data/signals/ema_pullback_signals.csv.

In [25]:
!python "/content/drive/MyDrive/trading-system/screeners/ema_pullback.py"


✅ Loaded 8 tickers from tickerlist.txt
Verifying project directories...
All directories are present in Google Drive.

--- Running EMA Trend Pullback Screener (with advanced columns) ---
Saving 8 signal(s) from ema_pullback to /content/drive/MyDrive/trading-system/data/signals/ema_pullback_signals.csv...
